In [3]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [4]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1
)

In [7]:
class Topic(BaseModel):     # 원하는 데이터 구조를 정의합니다.      
    description: str = Field(description="주제에 대한 간결한 설명")                     
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [8]:
question = "지구 온난화의 심각성 대해 알려주세요."

In [9]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)

In [10]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "주제에 대한 간결한 설명", "title": "Description", "type": "string"}, "hashtags": {"description": "해시태그 형식의 키워드(2개 이상)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

In [12]:
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

In [13]:
chain = prompt | llm | parser

In [14]:
answer = chain.invoke({"question": question})

In [15]:
print(answer)

{'description': '지구 온난화는 기후 변화로 인해 지구의 평균 온도가 상승하는 현상으로, 생태계와 인류에 심각한 영향을 미치고 있습니다.', 'hashtags': '#지구온난화 #기후변화'}


In [19]:
print(answer['description'])

지구 온난화는 기후 변화로 인해 지구의 평균 온도가 상승하는 현상으로, 생태계와 인류에 심각한 영향을 미치고 있습니다.


In [20]:
print(answer['hashtags'])

#지구온난화 #기후변화
